# Train by pre-trained model

### Download the pre-trained model online
<pre><font size = 4>
The lines below are the .config file in pre-trained model from

<a ref = https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models-coco-models>&#9658; Tensorflow detection model zoo</a>

Here we pick <b>ssd_inception_v2_coco</b> as our model and mark the lines we must fix to fit our data
</font></pre>

### Modify the .config in pre-trained model
<pre>
model {
  ssd {
    <span style='background:lightyellow'>num_classes: 90    <font color = "red"># Set this to the number of different label classes</font></span>
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    feature_extractor {
      <span style='background:lightyellow'>type: "ssd_inception_v2"    <font color = "red"># Set to the name of your chosen pre-trained model</font></span>
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.99999989895e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.0299999993294
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.999700009823
          center: true
          scale: true
          epsilon: 0.0010000000475
          train: true
        }
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      convolutional_box_predictor {
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 3.99999989895e-05
            }
          }
          initializer {
            truncated_normal_initializer {
              mean: 0.0
              stddev: 0.0299999993294
            }
          }
          activation: RELU_6
        }
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        dropout_keep_probability: 0.800000011921
        kernel_size: 3
        box_code_size: 4
        apply_sigmoid_to_scores: false
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.20000000298
        max_scale: 0.949999988079
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.333299994469
        reduce_boxes_in_lowest_layer: true
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 0.300000011921
        iou_threshold: 0.600000023842
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.990000009537
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 0
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
  }
}
train_config {
  <span style='background:lightyellow'>batch_size: 24    <font color = "red"># Increase/Decrease the value depending on memory (Higher values require more memory and vice-versa)</font></span>
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
  optimizer {
    rms_prop_optimizer {
      learning_rate {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.00400000018999
          decay_steps: 800720
          decay_factor: 0.949999988079
        }
      }
      momentum_optimizer_value: 0.899999976158
      decay: 0.899999976158
      epsilon: 1.0
    }
  }
  <span style='background:lightyellow'>fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED/model.ckpt"    <font color = "red"># Path to extracted files of pre-trained model</font></span>
  from_detection_checkpoint: true
    <font color = "red"># Note: The below line limits the training process to 200K steps, which we
    # empirically found to be sufficient enough to train the pets dataset. This
    # effectively bypasses the learning rate schedule (the learning rate will
    # never decay). Remove the below line to train indefinitely.</font>
  num_steps: 200000
}
train_input_reader {
  <span style='background:lightyellow'>label_map_path: "PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt"    <font color = "red"># Path to training TFRecord file</font></span>
  tf_record_input_reader {
    <span style='background:lightyellow'>input_path: "PATH_TO_BE_CONFIGURED/mscoco_train.record"    <font color = "red"># Path to label map file</font></span>
  }
}
eval_config {
  num_examples: 8000
  max_evals: 10
  use_moving_averages: false
}
eval_input_reader {
  <span style='background:lightyellow'>label_map_path: "PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt"    <font color = "red"># Path to testing TFRecord</font></span>
  shuffle: false
  num_readers: 1
  tf_record_input_reader {
    <span style='background:lightyellow'>input_path: "PATH_TO_BE_CONFIGURED/mscoco_val.record"    <font color = "red"># Path to label map file</font></span>
  }
}


<font size = 4 color = "red"><b>Note:</b> The path must use "/" instead of "\" if you use copy and paste to revise the path</font>
</pre>

### Get the train.py file
<pre>
<font size = 4>Head to <b>&#60;Path You Clone Models&#62;</b>/models/research/object_detection/legacy

Copy the <b>train.py</b> and paste to your workspace

Then back to prompt type
</font>
    <span style='background:#E8E8E8'>python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_inception_v2_coco.config</span>
    
<font size = 4>if you use the pre-trained-model <b>ssd_inception_v2_coco.config</b></font>
</pre>
